## Modeling (Part A)

## Loading Data

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import xgboost as xg 
import numpy as np 
from sklearn.model_selection import GridSearchCV

In [8]:
data = pd.read_csv("data/cleaned_data.csv")

data.head()

,Price,Shared Room,Private Room,Person Capacity,Superhost,Multiple Rooms,Business,Cleanliness Rating,Guest Satisfaction,Bedrooms,...,City_Budapest,City_Lisbon,City_Paris,City_Rome,City_Vienna,Day_Weekday,Day_Weekend,Room_Type_Entire home/apt,Room_Type_Private room,Room_Type_Shared room
0,194.033698,False,True,2.0,False,1,0,10.0,93.0,1,...,False,False,False,False,False,True,False,False,True,False
1,344.245776,False,True,4.0,False,0,0,8.0,85.0,1,...,False,False,False,False,False,True,False,False,True,False
2,433.529398,False,True,4.0,False,0,1,9.0,90.0,2,...,False,False,False,False,False,True,False,False,True,False
3,485.552926,False,True,2.0,True,0,0,10.0,98.0,1,...,False,False,False,False,False,True,False,False,True,False
4,552.808567,False,True,3.0,False,0,0,8.0,100.0,2,...,False,False,False,False,False,True,False,False,True,False


In [9]:
X = data.drop('Price', axis=1)
y = data['Price']

# Create bins for the price range based on quantiles for stratification
price_bins = pd.qcut(y, q=10, duplicates='drop') 

# Splitting the data into train, validation, and test sets
# 60% train, 20% validation, 20% test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42, stratify=price_bins)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=price_bins.loc[X_temp.index])


## Random Forest Regressor

In [10]:
# Initialize the Random Forest Regressor
rf_model = RandomForestRegressor(random_state=42)

# Train the model on the training set
rf_model.fit(X_train, y_train)

# Predict on the validation set
y_val_pred = rf_model.predict(X_val)

# Predict on the test set
y_test_pred = rf_model.predict(X_test)

# Calculate the Mean Squared Error on the test set
mse_test = mean_squared_error(y_test, y_test_pred)
rmse_test = np.sqrt(mse_test)

# Print the MSE and RMSE for the test set
print("Test MSE : %f" % mse_test) 
print("Test RMSE: %f" % rmse_test)

# Calculate the Mean Squared Error on the validation set
mse_val = mean_squared_error(y_val, y_val_pred)
rmse_val = np.sqrt(mse_val)
print("Validation MSE : %f" %(mse_val)) 
print("Validation RMSE: %f" % (rmse_val))

Test MSE : 4874.884542
Test RMSE: 69.820373
Validation MSE : 5277.217383
Validation RMSE: 72.644459


### Tuning Radom Forest

Doing a GridSearch in order to find the best parameters

In [12]:
# Define the parameter grid
param_grid = {
    'n_estimators': [100, 150, 200, 250]
}

# Initialize the grid search 
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, 
                           cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
best_params

Fitting 3 folds for each of 4 candidates, totalling 12 fits


{'n_estimators': 250}

Tunned Random Forest

In [15]:
# Initialize the Random Forest Regressor
rf_model_tuned = RandomForestRegressor(random_state=42, n_estimators= best_params['n_estimators'])

# Train the model on the training set
rf_model_tuned.fit(X_train, y_train)

# Predict on the validation set
y_val_pred = rf_model_tuned.predict(X_val)

# Predict on the test set
y_test_pred = rf_model_tuned.predict(X_test)

# Calculate the Mean Squared Error on the test set
mse_test = mean_squared_error(y_test, y_test_pred)
rmse_test = np.sqrt(mse_test)

# Print the MSE and RMSE for the test set
print("Test MSE : %f" % mse_test) 
print("Test RMSE: %f" % rmse_test)

# Calculate the Mean Squared Error on the validation set
mse_val = mean_squared_error(y_val, y_val_pred)
rmse_val = np.sqrt(mse_val)
print("Validation MSE : %f" %(mse_val)) 
print("Validation RMSE: %f" % (rmse_val))

Test MSE : 4825.438082
Test RMSE: 69.465373
Validation MSE : 5230.580310
Validation RMSE: 72.322751
